##  Task-Aware BPTT (Retropropagação no Tempo Sensível à Tarefa)

###  Contexto do Problema

Após a inicialização dos parâmetros (Seção IV.A), o próximo passo é o **ajuste fino** (fine tuning) do modelo FDRNN.  
Esse ajuste exige a aplicação do **método de retropropagação do erro**, mas com uma dificuldade adicional:

> A arquitetura FDRNN é **profunda e recorrente ao mesmo tempo**, o que torna o cálculo dos gradientes mais complexo do que em redes feedforward simples.

---

###  Gradiente com estrutura recorrente

Os autores indicam que, devido à recorrência, o cálculo do gradiente para qualquer parâmetro **envolve múltiplos termos recursivos**, incluindo dependências de estados anteriores ($δ_{t−1}$, $δ_{t−2}$, ..., até o tempo inicial).  

O gradiente da função de recompensa total $U_T$ com relação a um parâmetro $θ$ segue uma cadeia de dependência:



\begin{equation}
\frac{\partial U_T}{\partial \theta} = \sum_t \frac{d U_t}{d R_t} \left\{ 
\frac{d R_t}{d \delta_t} \cdot \frac{d \delta_t}{d \theta} 
+ \frac{d R_t}{d \delta_{t-1}} \cdot \frac{d \delta_{t-1}}{d \theta} 
\right\}
\end{equation}

\begin{equation}
\frac{d \delta_t}{d \theta} = \frac{\partial \delta_t}{\partial \theta} 
+ \frac{\partial \delta_t}{\partial \delta_{t-1}} \cdot \frac{d \delta_{t-1}}{d \theta}
\end{equation}


Isso gera uma **explosão de termos recursivos**, que torna o cálculo direto inviável e propenso a instabilidade.

---

###  Solução: Backpropagation Through Time (BPTT)

Para resolver isso, o artigo propõe usar a técnica conhecida como **Backpropagation Through Time (BPTT)**:

- O BPTT **"desenrola" a rede no tempo**, transformando a estrutura recorrente em uma sequência de redes feedforward.
- Cada "tempo" se torna uma **cópia da rede**, chamada de **time stack**.
- Com isso, a rede **perde sua recorrência temporariamente**, o que permite aplicar o algoritmo de backpropagation comum.

#### Exemplo visual:
- `τ = 0` → estado atual
- `τ = 1` → um passo anterior
- Cada `τ` é uma cópia da rede com seus próprios estados e camadas ocultas

---

###  Problema: Degradação do gradiente

Ao aplicar BPTT diretamente, surgem dois problemas:

1. A rede **fica ainda mais profunda**, já que se estende ao longo do tempo (tempo × profundidade = profundidade total maior).
2. O gradiente **tende a desaparecer** (vanishing gradient), especialmente:
   - Em camadas mais próximas da entrada
   - Em time stacks mais antigos (com `τ` alto)

Isso **prejudica o aprendizado**, porque os sinais de erro não chegam adequadamente às camadas que mais precisam ser ajustadas.

---

###  Proposta: Task-Aware BPTT

Para resolver esse problema, os autores propõem o **Task-Aware BPTT**:

- Durante o desenrolar no tempo, **linhas virtuais** são adicionadas, conectando diretamente:
  1. A **função objetivo `U_T`** com a saída de cada time stack
  2. A **saída de cada time stack** com suas respectivas camadas ocultas

#### O que isso muda?

- Cada bloco temporal **recebe gradientes diretamente da tarefa final** (`U_T`), além dos gradientes das etapas anteriores.
- Isso **reforça a retropropagação** e **reduz a perda de sinal de erro**, especialmente nas camadas mais distantes e nos tempos mais antigos.

---

###  Resumo da técnica

| Conceito                 | Explicação                                                             |
|--------------------------|------------------------------------------------------------------------|
| Time stack               | Cópia da rede para um instante no tempo (`τ`)                         |
| BPTT                     | Desenrola a recorrência para aplicar backpropagation padrão           |
| Problema do gradiente    | Em redes profundas e longas, o erro "some" antes de alcançar as bases |
| Task-aware BPTT          | Injeta gradiente diretamente da função de recompensa em cada tempo     |
| Benefício                | Aprendizado mais eficaz em todas as camadas e tempos                  |

---

###  Algoritmo 1 (resumo)

O algoritmo descrito no final da seção segue os seguintes passos:

1. Inicialize os parâmetros da FDRNN (já feito na Seção IV.A).
2. Desenrole a rede no tempo para formar os time stacks.
3. Calcule os gradientes em cada time stack, com:
   - Gradientes normais (da retropropagação padrão)
   - Gradientes adicionais vindos da função `U_T`
4. **Normalize o vetor de gradiente** (para evitar explosões numéricas).
5. Atualize os parâmetros com descida de gradiente.

---

###  Conclusão

O **Task-Aware BPTT** é uma adaptação sofisticada do BPTT tradicional, ajustada às necessidades de:

- Redes **profundas e recorrentes**
- Treinamento com **função de recompensa não supervisionada**
- Aplicações **financeiras ruidosas e sensíveis ao tempo**

Essa técnica permite **treinar a FDRNN de forma robusta**, mesmo com grande profundidade e dependência temporal.

Na próxima seção, o artigo testa esse modelo no mundo real.